# Práctica 1 Cómputo Evolutivo
**Problema del Viajero**

 Dada una lista de ciudades y la distancia entre cada par de ellas, en
contrar el camino más corto que visite cada ciudad exactamente una
 vez y regrese a la ciudad de origen.



**Consideraciones:**

* Etiquete a las ciudades como 0, 1, 2, ..., N-1.
* El viajero siempre parte de la ciudad n0.
* Tiene el mismo costo ir de la ciudad A a la ciudad B que ir de la ciudad B a la ciudad A.
* Una solución al problema es una permutación de las N ciudades.
* La solución inicial se genera utilizando un algoritmo voraz. Se parte de la ciudad n0, se revisa el costo de ir a las N-1 ciudades restantes y se elige la de menor costo. Posteriormente, se repite el proceso. Es importante considerar que estamos generando permutaciones y, por lo tanto, no puede haber ciudades repetidas en la solución.
* El vecindario de una solución se genera de la siguiente forma: Se elige al azar una posición de la permutación. Se generan N-2 soluciones nuevas moviendo la ciudad de esa posición a cualquiera de las otras N-2 posiciones posibles.
* En la lista tabú se almacena la ciudad que se utilizó para generar el vecindario y tendrá un tiempo tabú de N/2.
* La entrada al programa será:
    1.  La primera línea tendrá el número de ciudades N.
    2.  La segunda línea tendrá el número máximo de iteraciones Imax del algoritmo de BT.
    3.  Las siguientes N-1 líneas indicarán el costo de ir de una ciudad a otra. Es decir:
        * La primera línea tiene el costo de ir de la ciudad n0 a la ciudad n1, a la ciudad n2 y hasta la ciudad nN-1.
        * La segunda línea tiene el costo de ir de la ciudad n1 a la ciudad n2 y hasta la ciudad nN-1.
        * Así sucesivamente hasta llegar a la última línea que tiene el costo de ir de la ciudad nN-2 a la ciudad nN-1.
* La salida del programa será:
    1.  Recorrido que debe seguir el viajero (permutación).
    2.  Costo de seguir el recorrido encontrado.






In [ ]:
def convertir_lista_a_matriz(N, Imax, costos_lista):
    
    costos = [[0] * N for _ in range(N)] # Mariz de costos NxN inicializada en 0

    # Llenar la matriz con los valores dados (triangular superior)
    index = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            costos[i][j] = costos_lista[index]
            costos[j][i] = costos_lista[index]  # Para que sea simetrica
            index += 1  

    return costos
# Input
N = int(input().strip())  # Número de ciudades
Imax = int(input().strip())  # Num max de iteraciones
costos_lista = list(map(int, input().split()))  #  lista de costos 

matriz_costos = convertir_lista_a_matriz(N, Imax, costos_lista)


In [12]:
import random

def leer_entrada():
    N = int(input().strip())  # Número de ciudades
    Imax = int(input().strip())  # Iteraciones máximas
    costos_lista = list(map(int, input().split()))  # Leer lista de costos en una sola línea

    # Convertir lista a matriz simétrica de costos
    costos = [[0] * N for _ in range(N)]
    index = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            costos[i][j] = costos_lista[index]
            costos[j][i] = costos_lista[index]  # Matriz simétrica
            index += 1

    return N, Imax, costos

def heuristica_voraz(N, costos):
    visitadas = [0]  # Siempre empezamos en la ciudad 0
    while len(visitadas) < N:
        ultima = visitadas[-1]
        no_visitadas = [i for i in range(N) if i not in visitadas]
        siguiente = min(no_visitadas, key=lambda x: costos[ultima][x])
        visitadas.append(siguiente)
    
    return visitadas

def calcular_costo(ruta, costos):
    return sum(costos[ruta[i]][ruta[i+1]] for i in range(len(ruta) - 1)) + costos[ruta[-1]][ruta[0]]

def generar_vecindario(ruta, N):
    # Generar todos los vecinos posibles intercambiando dos ciudades
    vecinos = []
    for i in range(1, N):
        for j in range(i + 1, N):
            nueva_ruta = ruta[:]
            # Intercambiar las ciudades en las posiciones i y j
            nueva_ruta[i], nueva_ruta[j] = nueva_ruta[j], nueva_ruta[i]
            vecinos.append(nueva_ruta)
    return vecinos

def busqueda_tabu(N, Imax, costos):
    mejor_ruta = heuristica_voraz(N, costos)
    mejor_costo = calcular_costo(mejor_ruta, costos)
    
    lista_tabu = {}  # Diccionario {ciudad: iteración en la que sale de la lista}
    
    iteracion = 0
    while iteracion < Imax:
        vecinos = generar_vecindario(mejor_ruta, N)
        mejor_vecino = None
        mejor_vecino_costo = float('inf')
        
        # Buscar el mejor vecino
        for vecino in vecinos:
            costo = calcular_costo(vecino, costos)
            if costo < mejor_vecino_costo and all(ciudad not in lista_tabu for ciudad in vecino[1:]):  # Evitar ciudades en la lista tabú
                mejor_vecino = vecino
                mejor_vecino_costo = costo
        
        # Si encontramos un mejor vecino, actualizar la ruta
        if mejor_vecino and mejor_vecino_costo < mejor_costo:
            mejor_ruta = mejor_vecino
            mejor_costo = mejor_vecino_costo
            lista_tabu[mejor_vecino[0]] = iteracion + (N // 2)  # Tiempo tabú
        
        # Limpiar la lista tabú
        lista_tabu = {ciudad: tiempo for ciudad, tiempo in lista_tabu.items() if tiempo > iteracion}
        
        iteracion += 1
    
    return mejor_ruta, mejor_costo

# Lógica principal
N, Imax, costos = leer_entrada()
mejor_ruta, mejor_costo = busqueda_tabu(N, Imax, costos)

# Imprimir salida con formato corregido
print(" ".join(map(str, mejor_ruta)))  # Ruta con espacios
print(mejor_costo)  # Costo total


0 5 3 8 9 6 2 7 4 1
248


In [6]:
def leer_entrada():
    # Leer el número de ciudades
    N = int(input().strip())
    # Leer el número máximo de iteraciones
    Imax = int(input().strip())
    # Leer todos los costos en una sola línea
    costos = list(map(int, input().strip().split()))
    
    # Construir la matriz de costos simétrica
    matriz_costos = [[0] * N for _ in range(N)]
    index = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            matriz_costos[i][j] = costos[index]
            matriz_costos[j][i] = costos[index]  # Matriz simétrica
            index += 1
    
    return N, Imax, matriz_costos

def heuristica_voraz(N, costos):
    # Iniciar en la ciudad 0
    ruta = [0]
    # Mientras no se visiten todas las ciudades
    while len(ruta) < N:
        ultima_ciudad = ruta[-1]
        # Encontrar la ciudad no visitada más cercana
        no_visitadas = [i for i in range(N) if i not in ruta]
        siguiente_ciudad = min(no_visitadas, key=lambda x: costos[ultima_ciudad][x])
        ruta.append(siguiente_ciudad)
    return ruta

def calcular_costo(ruta, costos):
    # Calcular el costo total de la ruta
    costo = 0
    for i in range(len(ruta) - 1):
        costo += costos[ruta[i]][ruta[i + 1]]
    # Regresar a la ciudad de origen
    costo += costos[ruta[-1]][ruta[0]]
    return costo

def generar_vecindario(ruta, N):
    # Generar vecindario moviendo una ciudad a otra posición
    vecindario = []
    posicion = random.randint(1, N - 1)  # Elegir una posición al azar (excluyendo la ciudad 0)
    ciudad = ruta[posicion]
    for i in range(1, N):  # Mover la ciudad a otras posiciones (excluyendo la ciudad 0)
        if i != posicion:
            nueva_ruta = ruta[:]
            nueva_ruta.pop(posicion)
            nueva_ruta.insert(i, ciudad)
            vecindario.append(nueva_ruta)
    return vecindario

def busqueda_tabu(N, Imax, costos):
    # Paso 1: Generar solución inicial usando heurística voraz
    x0 = heuristica_voraz(N, costos)
    f0 = calcular_costo(x0, costos)
    
    # Paso 2: Inicializar la mejor solución encontrada
    xbest = x0[:]
    fbest = f0
    
    # Paso 3: Inicializar contador de iteraciones
    k = 0
    
    # Paso 4: Inicializar lista tabú
    lista_tabu = {}  # Diccionario {ciudad: iteración en la que sale de la lista}
    tiempo_tabu = N // 2  # Tiempo tabú
    
    # Paso 5: Bucle principal
    while k < Imax:
        # Paso 6: Generar vecindario
        vecindario = generar_vecindario(x0, N)
        
        # Paso 7: Filtrar vecindario para excluir soluciones tabú
        vecindario_reducido = []
        for vecino in vecindario:
            # Verificar si la ciudad movida está en la lista tabú
            ciudad_movida = vecino[1]  # La ciudad movida es la segunda en la ruta
            if ciudad_movida not in lista_tabu or lista_tabu[ciudad_movida] <= k:
                vecindario_reducido.append(vecino)
        
        # Paso 8: Seleccionar la mejor solución en el vecindario reducido
        mejor_vecino = None
        mejor_costo = float('inf')
        for vecino in vecindario_reducido:
            costo = calcular_costo(vecino, costos)
            if costo < mejor_costo:
                mejor_vecino = vecino
                mejor_costo = costo
        
        # Si no hay vecinos válidos, terminar
        if mejor_vecino is None:
            break
        
        # Paso 9: Actualizar la solución actual
        x0 = mejor_vecino[:]
        
        # Paso 10: Actualizar la mejor solución encontrada
        if mejor_costo < fbest:
            xbest = x0[:]
            fbest = mejor_costo
        
        # Paso 11: Actualizar la lista tabú
        ciudad_movida = mejor_vecino[1]  # La ciudad movida es la segunda en la ruta
        lista_tabu[ciudad_movida] = k + tiempo_tabu  # Agregar a la lista tabú
        
        # Paso 12: Incrementar contador de iteraciones
        k += 1
    
    # Paso 13: Regresar la mejor solución encontrada
    return xbest, fbest

# Lógica principal
import random

N, Imax, costos = leer_entrada()
mejor_ruta, mejor_costo = busqueda_tabu(N, Imax, costos)

# Imprimir salida
print(" ".join(map(str, mejor_ruta)))
print(mejor_costo)

0 5 3 8 9 6 2 7 1 4
271


In [12]:
def leer_entrada():
    # Leer el número de ciudades
    N = int(input().strip())
    # Leer el número máximo de iteraciones
    Imax = int(input().strip())
    # Leer todos los costos en una sola línea
    costos = list(map(int, input().strip().split()))
    
    # Construir la matriz de costos simétrica
    matriz_costos = [[0] * N for _ in range(N)]
    index = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            matriz_costos[i][j] = costos[index]
            matriz_costos[j][i] = costos[index]  # Matriz simétrica
            index += 1
    
    return N, Imax, matriz_costos
    print(matriz_costos)
def heuristica_voraz(N, costos):
    # Iniciar en la ciudad 0
    ruta = [0]
    # Mientras no se visiten todas las ciudades
    while len(ruta) < N:
        ultima_ciudad = ruta[-1]
        # Encontrar la ciudad no visitada más cercana
        no_visitadas = [i for i in range(N) if i not in ruta]
        siguiente_ciudad = min(no_visitadas, key=lambda x: costos[ultima_ciudad][x])
        ruta.append(siguiente_ciudad)
    return ruta

def calcular_costo(ruta, costos):
    # Calcular el costo total de la ruta
    costo = 0
    for i in range(len(ruta) - 1):
        costo += costos[ruta[i]][ruta[i + 1]]
    # Regresar a la ciudad de origen
    costo += costos[ruta[-1]][ruta[0]]
    return costo

def generar_vecindario(ruta, N):
    # Generar vecindario intercambiando dos ciudades
    vecindario = []
    for i in range(1, N):
        for j in range(i + 1, N):
            nueva_ruta = ruta[:]
            # Intercambiar ciudades en las posiciones i y j
            nueva_ruta[i], nueva_ruta[j] = nueva_ruta[j], nueva_ruta[i]
            vecindario.append(nueva_ruta)
    return vecindario

def busqueda_tabu(N, Imax, costos):
    # Paso 1: Generar solución inicial usando heurística voraz
    x0 = heuristica_voraz(N, costos)
    f0 = calcular_costo(x0, costos)
    
    # Paso 2: Inicializar la mejor solución encontrada
    xbest = x0[:]
    fbest = f0
    
    # Paso 3: Inicializar contador de iteraciones
    k = 0
    
    # Paso 4: Inicializar lista tabú
    lista_tabu = set()  # Conjunto de movimientos tabú
    tiempo_tabu = N // 2  # Tiempo tabú
    
    # Paso 5: Bucle principal
    while k < Imax:
        # Paso 6: Generar vecindario
        vecindario = generar_vecindario(x0, N)
        
        # Paso 7: Filtrar vecindario para excluir soluciones tabú
        vecindario_reducido = []
        for vecino in vecindario:
            # Verificar si el movimiento es tabú
            movimiento = tuple(sorted([vecino[1], vecino[2]]))  # Movimiento (ciudad1, ciudad2)
            if movimiento not in lista_tabu:
                vecindario_reducido.append(vecino)
        
        # Paso 8: Seleccionar la mejor solución en el vecindario reducido
        mejor_vecino = None
        mejor_costo = float('inf')
        for vecino in vecindario_reducido:
            costo = calcular_costo(vecino, costos)
            if costo < mejor_costo:
                mejor_vecino = vecino
                mejor_costo = costo
        
        # Si no hay vecinos válidos, terminar
        if mejor_vecino is None:
            break
        
        # Paso 9: Actualizar la solución actual
        x0 = mejor_vecino[:]
        
        # Paso 10: Actualizar la mejor solución encontrada
        if mejor_costo < fbest:
            xbest = x0[:]
            fbest = mejor_costo
        
        # Paso 11: Actualizar la lista tabú
        movimiento = tuple(sorted([mejor_vecino[1], mejor_vecino[2]]))  # Movimiento (ciudad1, ciudad2)
        lista_tabu.add(movimiento)
        if len(lista_tabu) > tiempo_tabu:
            lista_tabu.remove(next(iter(lista_tabu)))  # Eliminar el movimiento más antiguo
        
        # Paso 12: Incrementar contador de iteraciones
        k += 1
    
    # Paso 13: Regresar la mejor solución encontrada
    return xbest, fbest

# Lógica principal
import random

N, Imax, costos = leer_entrada()
mejor_ruta, mejor_costo = busqueda_tabu(N, Imax, costos)

# Imprimir salida
print(" ".join(map(str, mejor_ruta)))
print(mejor_costo)

0 5 3 8 9 6 2 7 4 1
248


In [13]:
def leer_entrada():
    # Leer el número de ciudades
    N = int(input().strip())
    # Leer el número máximo de iteraciones
    Imax = int(input().strip())
    # Leer todos los costos en una sola línea
    costos = list(map(int, input().strip().split()))
    
    # Construir la matriz de costos simétrica
    matriz_costos = [[0] * N for _ in range(N)]
    index = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            matriz_costos[i][j] = costos[index]
            matriz_costos[j][i] = costos[index]  # Matriz simétrica
            index += 1
    
    return N, Imax, matriz_costos

def heuristica_voraz(N, costos):
    # Iniciar en la ciudad 0
    ruta = [0]
    # Mientras no se visiten todas las ciudades
    while len(ruta) < N:
        ultima_ciudad = ruta[-1]
        # Encontrar la ciudad no visitada más cercana
        no_visitadas = [i for i in range(N) if i not in ruta]
        siguiente_ciudad = min(no_visitadas, key=lambda x: costos[ultima_ciudad][x])
        ruta.append(siguiente_ciudad)
    return ruta

def calcular_costo(ruta, costos):
    # Calcular el costo total de la ruta
    costo = 0
    for i in range(len(ruta) - 1):
        costo += costos[ruta[i]][ruta[i + 1]]
    # Regresar a la ciudad de origen
    costo += costos[ruta[-1]][ruta[0]]
    return costo

def generar_vecindario(ruta, N):
    # Generar vecindario intercambiando dos ciudades
    vecindario = []
    for i in range(1, N):
        for j in range(i + 1, N):
            nueva_ruta = ruta[:]
            # Intercambiar ciudades en las posiciones i y j
            nueva_ruta[i], nueva_ruta[j] = nueva_ruta[j], nueva_ruta[i]
            vecindario.append(nueva_ruta)
    return vecindario

def busqueda_tabu(N, Imax, costos):
    # Paso 1: Generar solución inicial usando heurística voraz
    x0 = heuristica_voraz(N, costos)
    f0 = calcular_costo(x0, costos)
    
    # Paso 2: Inicializar la mejor solución encontrada
    xbest = x0[:]
    fbest = f0
    
    # Paso 3: Inicializar contador de iteraciones
    k = 0
    
    # Paso 4: Inicializar lista tabú
    lista_tabu = set()  # Conjunto de movimientos tabú
    tiempo_tabu = N // 2  # Tiempo tabú
    
    # Paso 5: Bucle principal
    while k < Imax:
        # Paso 6: Generar vecindario
        vecindario = generar_vecindario(x0, N)
        
        # Paso 7: Filtrar vecindario para excluir soluciones tabú
        vecindario_reducido = []
        for vecino in vecindario:
            # Verificar si el movimiento es tabú
            movimiento = tuple(sorted([vecino[1], vecino[2]]))  # Movimiento (ciudad1, ciudad2)
            if movimiento not in lista_tabu:
                vecindario_reducido.append(vecino)
        
        # Paso 8: Seleccionar la mejor solución en el vecindario reducido
        mejor_vecino = None
        mejor_costo = float('inf')
        for vecino in vecindario_reducido:
            costo = calcular_costo(vecino, costos)
            if costo < mejor_costo:
                mejor_vecino = vecino
                mejor_costo = costo
        
        # Si no hay vecinos válidos, terminar
        if mejor_vecino is None:
            break
        
        # Paso 9: Actualizar la solución actual
        x0 = mejor_vecino[:]
        
        # Paso 10: Actualizar la mejor solución encontrada
        if mejor_costo < fbest:
            xbest = x0[:]
            fbest = mejor_costo
        
        # Paso 11: Actualizar la lista tabú
        movimiento = tuple(sorted([mejor_vecino[1], mejor_vecino[2]]))  # Movimiento (ciudad1, ciudad2)
        lista_tabu.add(movimiento)
        if len(lista_tabu) > tiempo_tabu:
            lista_tabu.remove(next(iter(lista_tabu)))  # Eliminar el movimiento más antiguo
        
        # Paso 12: Incrementar contador de iteraciones
        k += 1
    
    # Paso 13: Regresar la mejor solución encontrada
    return xbest, fbest

# Lógica principal
import random

N, Imax, costos = leer_entrada()
mejor_ruta, mejor_costo = busqueda_tabu(N, Imax, costos)

# Imprimir salida
print(" ".join(map(str, mejor_ruta)))
print(mejor_costo)

0 5 3 8 9 6 2 7 4 1
248


In [3]:
import random
import math

# Parámetros ajustables
Imax = 100  # Número máximo de iteraciones
tiempo_tabu = 10  # Tiempo tabú (en número de iteraciones)
n_ciudades = 10  # Número de ciudades
matriz_costos = [
    [0, 49, 30, 53, 72, 19, 76, 87, 45, 48],
    [49, 0, 19, 38, 32, 31, 75, 69, 61, 25],
    [30, 19, 0, 41, 98, 56, 6, 6, 45, 53],
    [53, 38, 41, 0, 52, 29, 46, 90, 23, 98],
    [72, 32, 98, 52, 0, 63, 90, 69, 50, 82],
    [19, 31, 56, 29, 63, 0, 60, 88, 41, 95],
    [76, 75, 6, 46, 90, 60, 0, 61, 92, 10],
    [87, 69, 6, 90, 69, 88, 61, 0, 82, 73],
    [45, 61, 45, 23, 50, 41, 92, 82, 0, 5],
    [48, 25, 53, 98, 82, 95, 10, 73, 5, 0]
]

# Función para generar una solución inicial utilizando el algoritmo voraz
def generar_solucion_inicial():
    ruta = [0]  # Comenzamos desde la ciudad 0
    ciudades_restantes = list(range(1, n_ciudades))
    
    while ciudades_restantes:
        ciudad_actual = ruta[-1]
        ciudad_proxima = min(ciudades_restantes, key=lambda x: matriz_costos[ciudad_actual][x])
        ruta.append(ciudad_proxima)
        ciudades_restantes.remove(ciudad_proxima)
    
    return ruta

# Función para calcular el costo de una ruta
def calcular_costo(ruta):
    costo = 0
    for i in range(len(ruta) - 1):
        costo += matriz_costos[ruta[i]][ruta[i + 1]]
    costo += matriz_costos[ruta[-1]][ruta[0]]  # Regresamos al punto de inicio
    return costo

# Función de búsqueda tabú
def busqueda_tabu():
    # Generar la solución inicial
    x = generar_solucion_inicial()
    fbest = calcular_costo(x)
    xbest = x[:]
    
    lista_tabu = set()
    k = 0
    
    # Parámetros para el enfriamiento simulado
    T_init = 1000  # Temperatura inicial
    T_min = 1  # Temperatura mínima
    alpha = 0.95  # Factor de enfriamiento

    while k < Imax:
        # Enfriamiento simulado (probabilidad de aceptación de peores soluciones)
        T = T_init * (alpha ** k)
        
        # Generar el vecindario (cambiar dos ciudades en la ruta)
        vecino = x[:]
        i, j = random.sample(range(1, n_ciudades), 2)  # Cambiar dos ciudades aleatorias
        vecino[i], vecino[j] = vecino[j], vecino[i]
        
        costo_vecino = calcular_costo(vecino)
        
        # Si el vecino es mejor o si se acepta una peor solución (dependiendo de la temperatura)
        if costo_vecino < fbest or random.random() < math.exp(-(costo_vecino - fbest) / T):
            x = vecino[:]
            fbest = costo_vecino
            xbest = x[:]
        
        # Añadir el movimiento al conjunto tabú
        lista_tabu.add((i, j))
        if len(lista_tabu) > tiempo_tabu:
            lista_tabu = set(list(lista_tabu)[1:])  # Eliminar el movimiento más antiguo
        
        k += 1
    
    return xbest, fbest

# Ejecutar la búsqueda tabú
solucion, costo = busqueda_tabu()
print("Mejor ruta encontrada:", solucion)
print("Costo de la mejor ruta:", costo)


Mejor ruta encontrada: [0, 5, 2, 6, 7, 8, 9, 1, 4, 3]
Costo de la mejor ruta: 391


In [8]:
import random

def calcular_costo(ruta, distancias):
    costo = 0
    n = len(ruta)
    for i in range(n):
        costo += distancias[ruta[i]][ruta[(i + 1) % n]]
    return costo

def generar_solucion_inicial(distancias):
    n = len(distancias)
    ruta = [0]  # Comenzamos desde la ciudad 0
    ciudades_restantes = list(range(1, n))
    
    while ciudades_restantes:
        ciudad_actual = ruta[-1]
        siguiente_ciudad = min(ciudades_restantes, key=lambda x: distancias[ciudad_actual][x])
        ruta.append(siguiente_ciudad)
        ciudades_restantes.remove(siguiente_ciudad)
    
    return ruta

def generar_vecindario(ruta, tabus):
    vecindario = []
    n = len(ruta)
    
    for i in range(1, n - 1):  # Evitamos la ciudad 0
        for j in range(1, n - 1):
            if i != j and (ruta[j] not in tabus):
                nueva_ruta = ruta[:]
                nueva_ruta[i], nueva_ruta[j] = nueva_ruta[j], nueva_ruta[i]  # Intercambiamos
                vecindario.append(nueva_ruta)
    
    return vecindario

def busqueda_tabu(distancias, Imax):
    n = len(distancias)
    mejor_ruta = generar_solucion_inicial(distancias)
    mejor_costo = calcular_costo(mejor_ruta, distancias)
    
    tabus = []
    
    for _ in range(Imax):
        vecindario = generar_vecindario(mejor_ruta, tabus)
        
        if not vecindario:
            break
        
        mejor_vecino = min(vecindario, key=lambda x: calcular_costo(x, distancias))
        costo_vecino = calcular_costo(mejor_vecino, distancias)
        
        if costo_vecino < mejor_costo:
            mejor_ruta = mejor_vecino
            mejor_costo = costo_vecino
        
        # Actualizar la lista tabú
        tabus.append(mejor_vecino[1])  # Agregamos la ciudad en la posición 1
        if len(tabus) > n // 2:
            tabus.pop(0)  # Mantenemos el tamaño de la lista tabú
        
    return mejor_ruta, mejor_costo

# Entrada de datos
N = int(input("Número de ciudades: "))
Imax = int(input("Número máximo de iteraciones: "))

# Inicializamos la matriz de distancias
print("Ingrese la lista de distancias en forma plana (separando los valores por espacios):")
distancias_flat = list(map(int, input().split()))

# Convertir la lista plana en una matriz simétrica
distancias = []
index = 0
for i in range(N):
    fila = []
    for j in range(N):
        if i == j:
            fila.append(0)  # La distancia a sí mismo es 0
        elif i < j:
            fila.append(distancias_flat[index])
            index += 1
        else:
            fila.append(distancias[j][i])  # Matriz simétrica
    distancias.append(fila)

# Ejecutamos la búsqueda tabú
ruta_final, costo_final = busqueda_tabu(distancias, Imax)

# Salida de resultados
print("Recorrido:", ruta_final)
print("Costo del recorrido:", costo_final)

Ingrese la lista de distancias en forma plana (separando los valores por espacios):
Recorrido: [0, 5, 3, 8, 9, 6, 2, 7, 1, 4]
Costo del recorrido: 271


In [ ]:
import random
from collections import deque

def leer_entrada():
    N = int(input().strip())  # Número de ciudades
    Imax = int(input().strip())  # Iteraciones máximas
    costos_lista = list(map(int, input().split()))  # Leer lista de costos en una sola línea

    # Convertir lista a matriz simétrica de costos
    costos = [[0] * N for _ in range(N)]
    index = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            costos[i][j] = costos_lista[index]
            costos[j][i] = costos_lista[index]  # Matriz simétrica
            index += 1

    return N, Imax, costos

def heuristica_voraz(N, costos):
    visitadas = [0]  # Siempre empezamos en la ciudad 0
    while len(visitadas) < N:
        ultima = visitadas[-1]
        no_visitadas = [i for i in range(N) if i not in visitadas]
        siguiente = min(no_visitadas, key=lambda x: costos[ultima][x])
        visitadas.append(siguiente)
    
    return visitadas

def calcular_costo(ruta, costos):
    return sum(costos[ruta[i]][ruta[i+1]] for i in range(len(ruta) - 1)) + costos[ruta[-1]][ruta[0]]

def generar_vecindario(ruta, N):
    # Generar vecinos intercambiando dos ciudades en la ruta
    for i in range(1, N):
        for j in range(i + 1, N):
            nueva_ruta = ruta[:]
            nueva_ruta[i], nueva_ruta[j] = nueva_ruta[j], nueva_ruta[i]
            yield nueva_ruta  # Usamos un generador para obtener los vecinos uno a uno

def busqueda_tabu(N, Imax, costos):
    mejor_ruta = heuristica_voraz(N, costos)
    mejor_costo = calcular_costo(mejor_ruta, costos)
    
    lista_tabu = deque()  # Usamos deque para la lista tabú
    lista_tabu_set = set()  # Usamos un set adicional para acelerar las verificaciones
    iteracion = 0
    while iteracion < Imax:
        mejores_vecinos = []
        
        # Generar y seleccionar los mejores vecinos
        for vecino in generar_vecindario(mejor_ruta, N):
            costo = calcular_costo(vecino, costos)
            
            # Condiciones para seleccionar un vecino:
            if costo < mejor_costo and not any(ciudad in lista_tabu_set for ciudad in vecino[1:]):
                mejores_vecinos.append((vecino, costo))
        
        if mejores_vecinos:
            # Seleccionamos el vecino con el costo más bajo
            mejor_vecino, mejor_vecino_costo = min(mejores_vecinos, key=lambda x: x[1])
            
            # Actualizar la mejor ruta y el costo
            mejor_ruta = mejor_vecino
            mejor_costo = mejor_vecino_costo
            
            # Actualizar la lista tabú con la ruta completa (no solo la primera ciudad)
            for ciudad in mejor_vecino:
                lista_tabu_set.add(ciudad)  # Añadimos todas las ciudades de la ruta a la lista tabú
            lista_tabu.append(mejor_vecino)  # Añadimos la ruta completa a la lista tabú
            
        # Limpiar la lista tabú: eliminamos la ciudad más antigua si ya ha pasado el tiempo
        if len(lista_tabu) > N // 2:
            ciudad_a_eliminar = lista_tabu.popleft()
            for ciudad in ciudad_a_eliminar:
                lista_tabu_set.remove(ciudad)  # También eliminamos de la versión set
        
        iteracion += 1
    
    return mejor_ruta, mejor_costo

# Lógica principal
N, Imax, costos = leer_entrada()
mejor_ruta, mejor_costo = busqueda_tabu(N, Imax, costos)

# Imprimir salida con formato corregido
print(" ".join(map(str, mejor_ruta)))  # Ruta con espacios
print(mejor_costo)  # Costo total


0 5 3 8 9 6 2 7 4 1
248


In [15]:
import random
from collections import deque

def leer_entrada():
    N = int(input().strip())  # # ciudades
    Imax = int(input().strip())  #  max iteraciones
    costos_lista = list(map(int, input().split()))  

    # costos a matriz
    costos = [[0] * N for _ in range(N)]
    index = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            costos[i][j] = costos[j][i] = costos_lista[index]
            index += 1

    return N, Imax, costos

def heuristica_voraz(N, costos):
    ruta = [0]  # ciudad 0
    while len(ruta) < N:
        actual = ruta[-1]
        no_visitadas = [ciudad for ciudad in range(N) if ciudad not in ruta]
        siguiente = min(no_visitadas, key=lambda ciudad: costos[actual][ciudad])
        ruta.append(siguiente)
    
    return ruta

def calcular_costo(ruta, costos):
    return sum(costos[ruta[i]][ruta[i+1]] for i in range(len(ruta) - 1)) + costos[ruta[-1]][ruta[0]]

def generar_vecindario(ruta, N):
    vecinos = []
    for i in range(1, N):
        for j in range(i + 1, N):
            vecino = ruta[:]
            vecino[i], vecino[j] = vecino[j], vecino[i]
            vecinos.append(vecino)  
    return vecinos

def busqueda_tabu(N, Imax, costos):
    mejor_ruta = heuristica_voraz(N, costos)
    mejor_costo = calcular_costo(mejor_ruta, costos)
    
    lista_tabu = deque()
    lista_tabu_set = set()
    
    for _ in range(Imax):
        mejores_vecinos = [
            (vecino, calcular_costo(vecino, costos))
            for vecino in generar_vecindario(mejor_ruta, N)
            if not any(ciudad in lista_tabu_set for ciudad in vecino[1:])
        ]

        if mejores_vecinos:
            mejor_ruta, mejor_costo = min(mejores_vecinos, key=lambda x: x[1])

        # Para agregar ciudades de la nueva ruta a la lista tabú
            lista_tabu.append(mejor_ruta)
            lista_tabu_set.update(mejor_ruta)

        # tiempo tabu
        if len(lista_tabu) > N // 2:
            lista_tabu_set.difference_update(lista_tabu.popleft())

    return mejor_ruta, mejor_costo


N, Imax, costos = leer_entrada()
mejor_ruta, mejor_costo = busqueda_tabu(N, Imax, costos)

# para espacios
print(" ".join(map(str, mejor_ruta)))
print(mejor_costo)


0 5 3 8 9 6 2 7 4 1
248
